In [3]:
pwd

'/root/eks-ml-pipeline'

In [5]:
!pip install /root/msspackages/dist/msspackages-0.0.7-py3-none-any.whl

Keyring is skipped due to an exception: 'keyring.backends'
Processing /root/msspackages/dist/msspackages-0.0.7-py3-none-any.whl
  Using cached configparser-5.3.0-py3-none-any.whl (19 kB)
  Using cached pyspark-3.3.1-py2.py3-none-any.whl
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)


In [6]:
from msspackages import setup_runner
setup_runner()

b"Get:1 http://security.debian.org/debian-security buster/updates InRelease [34.8 kB]\nGet:2 http://deb.debian.org/debian buster InRelease [122 kB]\nGet:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]\nGet:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [405 kB]\nGet:5 http://deb.debian.org/debian buster/main amd64 Packages [7909 kB]\nGet:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [8788 B]\nFetched 8536 kB in 2s (5558 kB/s)\nReading package lists...\nBuilding dependency tree...\nReading state information...\n62 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nThe following NEW packages will be installed:\n  sudo\n0 upgraded, 1 newly installed, 0 to remove and 62 not upgraded.\nNeed to get 1244 kB of archives.\nAfter this operation, 3883 kB of additional disk space will be used.\nGet:1 http://security.debian.org/

In [ ]:
from feature_engineering import node_autoencoder_ad_preprocessing, node_autoencoder_ad_feature_engineering, node_autoencoder_train_test_split
import pandas as pd

#pre processing
node_features_data, node_processed_data = node_autoencoder_ad_preprocessing("node_autoencoder_ad","11-21-2022","2022","10","10","10","128gb")

#test, train split
node_train_data, node_test_data = node_autoencoder_train_test_split(node_processed_data)

#Train data feature engineering
node_training_data, node_training_tensor = node_autoencoder_ad_feature_engineering(node_features_data, node_train_data)
print(node_training_data.show(truncate=False))
print(node_training_tensor.shape)

#Test data feature engineering
# node_testing_data, node_testing_tensor = node_autoencoder_ad_feature_engineering(node_features_data, node_test_data)
# print(node_testing_data.show(truncate=False))
# print(node_testing_tensor.shape)

In [ ]:
from feature_engineering import container_autoencoder_ad_preprocessing, container_autoencoder_ad_feature_engineering, container_autoencoder_train_test_split
import pandas as pd

#pre processing
container_features_data, container_processed_data = container_autoencoder_ad_preprocessing("container_autoencoder_ad","11-21-2022","2022","10","10","10","128gb")

#test, train split
container_train_data, container_test_data = container_autoencoder_train_test_split(container_processed_data)

#Train data feature engineering
container_training_data, container_training_tensor = container_autoencoder_ad_feature_engineering(container_features_data, container_train_data)
print(node_training_data.show(truncate=False))
print(container_training_tensor.shape)

#Test data feature engineering
# container_testing_data, container_testing_tensor = container_autoencoder_ad_feature_engineering(container_features_data, container_test_data)
# print(node_testing_data.show(truncate=False))
# print(container_testing_tensor.shape)